In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
url = 'https://www.cars.com/shopping/results/?dealer_id=&keyword=&list_price_max=&list_price_min=&makes[]=&maximum_distance=150&mileage_max=100000&page_size=100&sort=best_match_desc&stock_type=all&year_max=&year_min=&zip=55423'
page = requests.get(url)
page

<Response [200]>

In [3]:
soup = BeautifulSoup(page.text, 'lxml')

In [4]:
columns = ['Year','Make','Model','Used_or_New','Price','ConsumerRating','ConsumerReviews','SellerType','SellerName','SellerRating','SellerReviews',
           'StreetName','State','Zipcode','DealType','ComfortRating','InteriorDesignRating','PerformanceRating','ValueForMoneyRating',
           'ExteriorStylingRating','ReliabilityRating','ExteriorColor','InteriorColor','Drivetrain','MinMPG','MaxMPG','FuelType',
           'Transmission','Engine','VIN','Stock','Mileage']   

df= pd.DataFrame(columns=columns)

In [5]:
for i in range(12):
    
    postings = soup.find_all('a', class_ = 'vehicle-card-link js-gallery-click-link')
    current_page = soup
    for post in postings:
        try:
            link = 'http://cars.com' + post.get('href')
            info_page = requests.get(link)
            soup = BeautifulSoup(info_page.text, 'lxml')
            
            ###DELETE THIS WHEN YOURE DONE
            #test_link = 'https://www.cars.com/vehicledetail/1d26a033-1886-4e9c-80cb-fb4f15ff4d51/'
            #info_page = requests.get(test_link)
            #soup = BeautifulSoup(info_page.text, 'lxml')
            
            
            #Data we want
            title_info = soup.find('h1', class_ = 'listing-title').text.split()
            year = title_info[0]
            make = title_info[1]
            model = ' '.join(title_info[2:])
            used_or_new = soup.find('p', class_ = 'new-used').text
            price = soup.find('span', class_ = 'primary-price').text
            consumer_rating = float(soup.find('section', class_= 'sds-page-section vehicle-reviews').find('span', class_='sds-rating__count').text)
            consumer_reviews = int(soup.find('section',class_= 'sds-page-section vehicle-reviews').find('a',class_= 'sds-rating__link sds-button-link').text[1:].split()[0])
            
            #Seller Details
            #Dealer
            try:
                seller_name = soup.find('h3', class_ = 'sds-heading--5 heading seller-name').text
                seller_rating = float(soup.find('section',class_ = 'sds-page-section seller-info').find('span', class_ = 'sds-rating__count').text)
                seller_reviews = soup.find('a', class_ = 'sds-rating__link sds-button-link').text.split()[0][1:]
                seller_type = 'Dealer'
                if ',' in seller_reviews:
                    seller_reviews = int(''.join(seller_reviews.split(',')))
                else:
                    seller_reviews =  int(seller_reviews)
                address_pieces = soup.find('div', class_ = 'dealer-address').text.split(',')
                street_name = address_pieces[0]
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
                
            #Private seller    
            except:
                seller_type = 'Private'
                pieces = soup.find('div', class_ = 'seller-address').text.split('from')
                seller_name = pieces[0].strip()
                address_pieces = pieces[1].split(',')
                street_name = address_pieces[0].strip()
                state = address_pieces[1].split()[0]
                zipcode = address_pieces[1].split()[1] 
    
            
            #Deal type
            deal_type = 'NA'
            first_badge = soup.find('span', class_ = 'sds-badge__label').text
            if 'deal' in first_badge.casefold():
                deal_type = first_badge.split()[0]
            
            #Rating breakdown
            rb = soup.find_all('div', class_ = 'review-breakdown')[0].find_all('span', class_ = 'sds-definition-list__value')
            
            comfort = float(rb[0].text)
            interior_design = float(rb[1].text)
            performance = float(rb[2].text) 
            value_for_money = float(rb[3].text)
            exterior_styling = float(rb[4].text)
            reliability = float(rb[5].text)
            
            #Basic Description Table
            bd = soup.find_all('div', class_ = 'basics-content-wrapper')[0].find_all('dd')
            
            exterior_color = bd[0].text.strip()
            interior_color = bd[1].text.strip()
            drive_train = bd[2].text.strip()
            min_mpg = int(bd[3].text.split('\n')[2].split('–')[0])
            max_mpg = int(bd[3].text.split('\n')[2].split('–')[1])
            fuel_type = bd[4].text.strip()
            transmission = bd[5].text.strip()
            engine = bd[6].text.strip()
            vin = bd[7].text.strip()
            stock_number = bd[8].text.strip()
            mileage = int(''.join(bd[9].text.split()[0].split(',')))
            
            
    
            
            
            
            new_row = pd.DataFrame([[year,make,model,used_or_new,price,consumer_rating,consumer_reviews,seller_type,seller_name,seller_rating,seller_reviews,
                                   street_name,state,zipcode,deal_type,comfort,interior_design,performance,value_for_money,exterior_styling,reliability,
                                   exterior_color,interior_color,drive_train,min_mpg,max_mpg,fuel_type,transmission,engine,vin,stock_number,mileage]],
                                  columns=columns)
            
            #Generate row in dataframe based on scraped information
            df = pd.concat([df,new_row])
        except:
            pass
    
    current_page    
    current_page.find('a', {'aria-label':'Next page'}).get('href')
    next_page = 'http://cars.com' + current_page.find('a', {'aria-label':'Next page'}).get('href') #Get link to next page
        
    url = next_page
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'lxml')
    print(i)

#file = 'C:/Users/leave/OneDrive/Documents/Scraped_Datasets/Cars_Data/cars_raw.csv'
file = 'C:/Users/John/Desktop/BOOTCAMP/Car_Search_Project'

df.to_csv("cars_raw.csv")
#df.to_excel("cars_raw.xlsx") 

0
1
2
3
4
5
6
7
8
9
10
11


In [6]:
df

,Year,Make,Model,Used_or_New,Price,ConsumerRating,ConsumerReviews,SellerType,SellerName,SellerRating,...,InteriorColor,Drivetrain,MinMPG,MaxMPG,FuelType,Transmission,Engine,VIN,Stock,Mileage
0,2019,Audi,Q8 3.0T Premium,Used,"$64,450",4.8,27,Dealer,Motorwerks BMW,4.7,...,Black,All-wheel Drive,17,22,Gasoline,8-Speed Automatic,3.0L V6 24V GDI DOHC Turbo,WA1EVAF19KD043058,BN29678T,33621
0,2019,BMW,M550 i xDrive,Used,"$59,150",4.5,3,Dealer,Motorwerks BMW,4.7,...,Ivory White,All-wheel Drive,18,25,Gasoline,8-Speed Automatic,4.4L V8 32V GDI DOHC Twin Turbo,WBAJB9C59KB288941,BM96118T,33973
0,2019,Audi,Q5 2.0T Premium Plus,Used,"$43,348",4.7,42,Dealer,Audi Richfield,4.1,...,Rock Gray / Granite Gray,All-wheel Drive,22,27,Gasoline,7-Speed Automatic with Auto-Shift,2.0L I4 16V GDI DOHC Turbo,WA1BNAFY9K2087671,K2087671,16500
0,2019,Toyota,RAV4 Limited,Used,"$40,899",4.6,444,Dealer,Walser Toyota,4.6,...,Black,All-wheel Drive,25,33,Gasoline,Automatic,Regular Unleaded I-4 2.5 L/152,JTMN1RFV1KD503638,3BK634T,10718
0,2022,Genesis,GV70 3.5T Sport,Used,"$70,490",5.0,2,Dealer,Luther Bloomington Hyundai,4.7,...,Red,All-wheel Drive,19,25,Gasoline,8-Speed Automatic,3.5L V6 24V PDI DOHC Twin Turbo,KMUMCDTCXNU040735,2344A,12190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,2019,Ford,F-150 XLT,Used,"$38,950",4.8,425,Dealer,Diamond Buick GMC of Alexandria,4.6,...,Medium Earth Gray,Four-wheel Drive,18,23,Gasoline,10-Speed Automatic,2.7L V6 24V PDI DOHC Twin Turbo,1FTEW1EP2KKC93729,C93729P,41821
0,2019,Chevrolet,Traverse LT Leather,Used,"$35,000",4.7,203,Dealer,Decorah Chevrolet Decorah Chrysler,4.2,...,Gray,All-wheel Drive,17,25,Gasoline,9-Speed Automatic,3.6L V6 24V GDI DOHC,1GNEVHKW0KJ123230,123230,76468
0,2021,Jeep,Cherokee Latitude Lux,Used,"$31,904",4.9,71,Dealer,Valu Ford Chrysler,4.8,...,Ski Gray,Four-wheel Drive,19,27,Gasoline,9-Speed Automatic,3.2L V6 24V MPFI DOHC,1C4PJMMX9MD107749,F107749,30471
0,2017,RAM,1500 Laramie,Used,"$35,999",4.7,496,Dealer,Valu Ford Chrysler,4.8,...,Black,Four-wheel Drive,16,23,Diesel,8-Speed Automatic,3.0L V6 24V DDI DOHC Turbo Diesel,1C6RR7NMXHS874560,F874560A,71777
